In [14]:
import os
import re

import pandas as pd

In [1]:
class Modeling_rule_based:
    # get product name manual
    ## filter product name, perlu di regex lagi
    def get_product(self, p):
        p = p[p.find('*'):len(p)]
        p = p[:p.find('\n')]
        p = p.lower()
        p = re.sub("[^0-9a-z]+", " ", p)

        replacements = {
            '(?<=\s|[0-9])+(g|gr)+(?=\s|\.|$)' : 'gram',
            '(?<=\s|[0-9])+mg+(?=\s|\.|$)' : 'miligram',
            '(?<=\s|[0-9])+ml+(?=\s|\.|$)' : 'mililiter',
            '(?<=\s|[0-9])+l+(?=\s|\.|$)' : 'liter'
        }

        for key, value in replacements.items():
            p = re.sub(key, value, p)

        # p = re.sub("gr",  "gram", p)
        # p = re.sub("ml", "mililiter", p)
        return p

    def product_name_transform(self, str):
        str = re.sub("(?<=[A-Za-z])+[0-9]+(?=\s|\.|$)",
                    lambda group: " "+group[0], str)
        return str

    # create question manual
    ## keyword berdasarkan huruf terbanyak yang muncul
    def create_questions(self, c, q):
        list_q = []
        if c == "product_info":
            q1 = q 
            q2 = "kegunaan " + q
            q3 = "penggunaan " + q
            q4 = "cara pakai " + q
            q5 = "bagaimana menggunakan " + q
            q6 = "cara menggunakan " + q
            q7 = "cara penyimpanan " + q
            q8 = "bagaimana menyimpan " + q
            q9 = "cara menyimpan " + q
            q10 = "apa itu " + q
            q11 = "harga " + q
            q12 = "berapa biaya " + q
            list_q = [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12]
        elif c == "treatment_info":
            q1 = q
            q2 = "apa itu " + q
            q3 = "manfaat " + q
            q4 = "manfaat menggunakan" + q
            q5 = "keuntungan menggunakan " + q
            q6 = "prosedur " + q
            q7 = "bagaimana pengerjaan " + q
            q8 = "bagaimana cara kerja " + q
            q9 = "durasi " + q
            q10 = "berapa lama " + q
            q11 = "harga " + q
            q12 = "berapa biaya " + q
            list_q = [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12]
        
        return list_q

## Creating Question Pattern

In [2]:
model_rule = Modeling_rule_based()

In [5]:
os.chdir("E:\\Documents\\Coding\\Notebooks\\Chatbot\\apotek")
print(os.getcwd())

E:\Documents\Coding\Notebooks\Chatbot\apotek


In [8]:
df = pd.read_csv("dataset_transformation - dataset_transformation.csv")

In [10]:
df.head()

,Content ID,Answer 1,answer_tokenized,Status question,Next Event,Next Data,Version ID,Condition 1 Operator,Condition 1 Property,Condition 1 Value
0,conversation_start,"Hi. Welcome to ERHA, \n\nTerima kasih telah me...","['hi', 'welcome', 'to', 'erha', 'terima', 'kas...",TRUE,goto,conversation_start:user_info,NaN,NaN,NaN,NaN
1,fallback_too_short,"Keyword yang dimasukkan tidak Elsa mengerti, s...","['keyword', 'elsa', 'erti', 'silah', 'ketik', ...",TRUE,NaN,NaN,NaN,INCLUDES,source,whatsapp
2,fallback_too_complex,"Keyword yang dimasukkan tidak Elsa mengerti, s...","['keyword', 'elsa', 'erti', 'silah', 'ketik', ...",TRUE,NaN,NaN,NaN,INCLUDES,source,whatsapp
3,fallback_too_complex_simplify,"Keyword yang dimasukkan tidak Elsa mengerti, s...","['keyword', 'elsa', 'erti', 'silah', 'ketik', ...",TRUE,NaN,NaN,NaN,INCLUDES,source,whatsapp
4,handover_livechat_agent_end,Terima kasih sudah menghubungi ERHA,"['terima', 'kasih', 'hubung', 'erha']",TRUE,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df2 = df[df['Content ID'].isin(['product_info','treatment_info'])][['Content ID', 'Version ID', 'Answer 1']]
df2

,Content ID,Version ID,Answer 1
50,product_info,erha4,Berikut Informasi mengenai *Moisturizer ERHA 4...
51,product_info,erha5,Berikut Informasi mengenai *Erha 5 Evening Moi...
52,product_info,erha6,Berikut Informasi mengenai *Erha 6 Evening Moi...
54,product_info,erha7,Berikut Informasi mengenai *Erha 7 Foundation*...
55,product_info,erha9,Berikut Informasi mengenai *Erha 9 Foundation*...
...,...,...,...
552,treatment_info,BLACK CARBON SKINPROVEMENT LASER,_Black Carbon Skinprovement Laser_ adalah tind...
553,product_info,Acne Guard Cream,"HISERHA Acne Guard Cream, merupakan krim pelem..."
554,product_info,HISERHA ENERGY BRIGHT CREAM,"HISERHA Energy Bright Cream, merupakan krim pe..."
555,product_info,Urban Skin Defense Multi Benefit Refreshing Spray,ERHA PERFECT SHIELD URBAN SKIN DEFENSE Multi B...


In [12]:
df2['Version ID'] = df2['Version ID'].map(lambda x : str(x).lower())

In [15]:
question_pattern_list = []

for c, q in zip(df2['Content ID'], df2['Version ID']):
    question_pattern_list.append(model_rule.create_questions(str(c), model_rule.product_name_transform(str(q).lower()).strip()))

df2['Question Pattern'] = question_pattern_list
df2

,Content ID,Version ID,Answer 1,Question Pattern
50,product_info,erha4,Berikut Informasi mengenai *Moisturizer ERHA 4...,"[erha 4, kegunaan erha 4, penggunaan erha 4, c..."
51,product_info,erha5,Berikut Informasi mengenai *Erha 5 Evening Moi...,"[erha 5, kegunaan erha 5, penggunaan erha 5, c..."
52,product_info,erha6,Berikut Informasi mengenai *Erha 6 Evening Moi...,"[erha 6, kegunaan erha 6, penggunaan erha 6, c..."
54,product_info,erha7,Berikut Informasi mengenai *Erha 7 Foundation*...,"[erha 7, kegunaan erha 7, penggunaan erha 7, c..."
55,product_info,erha9,Berikut Informasi mengenai *Erha 9 Foundation*...,"[erha 9, kegunaan erha 9, penggunaan erha 9, c..."
...,...,...,...,...
552,treatment_info,black carbon skinprovement laser,_Black Carbon Skinprovement Laser_ adalah tind...,"[black carbon skinprovement laser, apa itu bla..."
553,product_info,acne guard cream,"HISERHA Acne Guard Cream, merupakan krim pelem...","[acne guard cream, kegunaan acne guard cream, ..."
554,product_info,hiserha energy bright cream,"HISERHA Energy Bright Cream, merupakan krim pe...","[hiserha energy bright cream, kegunaan hiserha..."
555,product_info,urban skin defense multi benefit refreshing spray,ERHA PERFECT SHIELD URBAN SKIN DEFENSE Multi B...,[urban skin defense multi benefit refreshing s...


## Creating JSON

In [16]:
def unit_transform(str):
    replacements = {
            '[0-9]+,' : '.',
        }

    for key, value in replacements.items():
        str = re.sub(key, value, str)
    return (str)

In [17]:
def generate_json(df):
    contexts = df['Question Pattern']
    tags = df['Version ID']
    responses = df['Answer 1']
    result = "{\"intents\": ["

    for context, tag, response in zip(contexts, tags, responses):
        result = result + "{\"context\": \"\",\"patterns\": " + str(context) + ",\"responses\": "+ "[\""+ str(response) +"\"]" +",\"tag\": \""+ str(tag) +"\"},"

    result = r"{}".format(result)
    result = result.replace("\'", "\"")
    result = result.replace("\'", "\"")
    # result = result.replace("“", r"\"")
    # result = result.replace("”", r"\"")
    # result = result.replace("–", r"-")

    return unit_transform(result) + "]}"

In [18]:
f = open("data-2.json", "w")
f.write(str(generate_json(df2).encode('ascii', 'replace')))
f.close()